In [ ]:
# def create_dict_trans(t):
#     dict_trans = {}
#     for trans in t:
#         if len(trans.split('<-')) == 2:
#             el1 = trans.split('<-')[0].strip()
#             el2 =  trans.split('<-')[1].strip()
#             if el1 not in dict_trans:
#                 dict_trans[el1] = [el2]
#             else:
#                 new_trans = dict_trans[el1]
#                 new_trans.append(el2)
#                 dict_trans[el1] = new_trans
#     return dict_trans

In [235]:
import string
import re

def find_alph(t):
    alphabet = ""
    uppercase_letters = string.ascii_uppercase
    for i, trans in enumerate(t):
        trans.split('<-')
        if len(trans) > 1:
            alphabet += uppercase_letters[i]
    return alphabet

def remove_operators_and_numbers(input_string):
    # Use a regular expression to remove operators and numbers
    result_string = re.sub(r'[0-9+\-*/\s]', '.', input_string)
    result_string = result_string.split('.')
    return [item for item in result_string if item != '']

def create_list_trans(t):
    list_trans = []
    for trans in t:
        if len(trans.split('<-')) == 2:
            el1 = trans.split('<-')[0].strip()
            el2 =  trans.split('<-')[1].strip()
            list_trans.append((el1, remove_operators_and_numbers(el2)))
    return list_trans

def find_D(alph, list_t):
    D = [[] for _ in range(len(alph))]
    for i, let in enumerate(alph):
        args = list_t[i][1]
        for arg in args:
            for j, l in enumerate(alph):
                if arg in list_t[j][0]: 
                    if alph[j] not in D[i]: D[i].append(alph[j])
                    if alph[i] not in D[j]: D[j].append(alph[i])
    return D

def find_I(alph, D):
    I = [list(alph) for i in range(len(alph))]

    for i in range(len(I)):
         I[i] = [x for x in I[i] if x not in D[i]]
    
    return I



def calc_foata(word, alph, D, helping_prints = False):
    if helping_prints:print("\n")

    dict_alph = {}
    for i,let in enumerate(alph):
        dict_alph[let] = i
    
    # create graph
    graph = [[] for _ in range(len(word))]
    graph_letters = [[] for _ in range(len(word))]
    for i, letter in enumerate(word):
        if letter not in alph: raise Exception("Letter not in alphabet")
        for j in range(i, len(word)):
            if i != j and word[j] in D[dict_alph[letter]]:
                graph_letters[i].append(word[j])
                graph[i].append(j)
    if helping_prints:
        print(f"Graph indexes:\n\t{graph}")
        print(f"Graph letters:\n\t{graph_letters}")


    # use changed bfs to define foata norm
    vals = [-1 for _ in range(len(word))]
    round_nr = 0
    start_point = 0
    queue = [(start_point, round_nr, start_point)]
    deleted_verts = []
    while queue:
        current_node, round_nr, parent_node = queue.pop(0)
        # if vals[current_node] < round_nr and vals[current_node] != -1: deleted_verts.append((parent_node, current_node))
        vals[current_node] = round_nr
        for el in graph[current_node]:
            queue.append((el,round_nr+1, current_node))
    if helping_prints:print(f"Values after bfs-like algorithm:\n\t{vals}")

    # create graph without long-distance vertices
    if helping_prints:print(f"Deleted verts are:\n\t{deleted_verts}")

    # calculate foata norm
    curr_val = 0
    max_val = max(vals)
    res_tab = [[] for _ in range((max_val+1))]
    res_tab_letters = [[] for _ in range((max_val+1))]
    while curr_val <= max_val:
        for i,el in enumerate(vals):
            if curr_val == vals[i]:
                res_tab[curr_val].append(i)
                res_tab_letters[curr_val].append(word[i])
        curr_val += 1
    res_tab_letters = [sorted(el) for el in res_tab_letters]
    return res_tab, res_tab_letters


if __name__ == "__main__":
    word = 'BAADCB'  # Sample word
    transactions = ['x<-x+y', 'y<-y+2z','x<-3x+z','z<-y-z']
    show_prints = True

    list_transaction = create_list_trans(transactions)
    if show_prints: print(f"Transactions dependencies:\n\t{list_transaction}")

    alphabet = find_alph(transactions)
    if show_prints: print(f"Alphabet is:\n\t{alphabet}")    
    D = find_D(alphabet, list_transaction)
    if show_prints: print(f"D is:\n\t{D}")
    I = find_I(alphabet, D)
    if show_prints: print(f"I is:\n\t{I}")

    foata_norm, foata_norm_letters = calc_foata(word, alphabet, D, True)
    if show_prints: print(f"foata norm is:\n\t{foata_norm_letters}")




Transactions dependencies:
	[('x', ['x', 'y']), ('y', ['y', 'z']), ('x', ['x', 'z']), ('z', ['y', 'z'])]
Alphabet is:
	ABCD
D is:
	[['A', 'C', 'B'], ['A', 'B', 'D'], ['A', 'C', 'D'], ['B', 'C', 'D']]
I is:
	[['D'], ['C'], ['B'], ['A']]


Graph indexes:
	[[1, 2, 3, 5], [2, 4, 5], [4, 5], [4, 5], [], []]
Graph letters:
	[['A', 'A', 'D', 'B'], ['A', 'C', 'B'], ['C', 'B'], ['C', 'B'], [], []]
Values after bfs-like algorithm:
	[0, 1, 2, 1, 3, 3]
Deleted verts are:
	[]
foata norm is:
	[['B'], ['A', 'D'], ['A'], ['B', 'C']]
